# 1. 频繁模式挖掘——Apriori和FP-Growth

# 2. 实验目标

掌握频繁模式挖掘的一些定义，知道如何运用$Python$计算支持度，置信度等

掌握mlxtend库调用Apriori算法和FP-Growth算法

# 3. 实验环境
|名称|版本|
|----|----|
|Pandas|1.1.3|
|mlxtend|0.19.0|

# 4. 实验要求

独立完成

# 5. 实验内容

## 5.1 理论
### 5.1.1 频繁模式挖掘
常用于发现大量数据中有意义的联系，这种联系强调的是产生数据的行为之间的联系。如在频繁模式挖掘中最为经典的“啤酒-尿布”购物篮分析的例子，我们就期望可以购买不同商品之间的联系，如我们发现很多顾客买了尿布的同时还会买啤酒，即产生如下的数据关联：

{尿布}$\rightarrow ${啤酒}

上述数据之间的关联就是所谓的关联规则。该规则或模式表明尿布和啤酒之间存在很强的联系，或许很多消费者在购买尿布的同时也会购买啤酒。作为商家可以利用这种规则发现新的商机。但是注意本此实验讨论的数据之间的关联是指这些数据会同时出现，但不讨论他们之间的因果关系。

几个在频繁模式挖掘中常见的关键词
- **项集**：项的集合。若一个项集包含$k$个项，则称其为“$k$-项集”，$k$为项集的长度。比如{$bread,milk$}就是一个$2$-项集。（空集是不含任何项的项集）
- **关联规则**：形如$X\rightarrow Y$的蕴含表达式，其中$X$和$Y$是不相交的项集。关联规则的强度可以用于支持度和置信度来衡量，同时满足最小支持度和执行度的关联规则称为**强关联规则**
- **支持度计数**：指定项集出现的频数
- **支持度**：指定项集所占的比例 $$s(X\rightarrow Y)=\frac{Count(X,Y)}{N}$$
- **置信度**：$Y$在包含$X$的交易中出现的频繁程度，其定义为：$$c(X\rightarrow Y)=\frac{Count(X,Y)}{Count(X)}$$
- **频繁项集**：支持度大于等于所设定的阈值的项集，如果频繁项集中有$L$项，记为$L$-频繁项集

### 5.1.2 Apriori算法
是一种频繁项集算法，其两个输入参数分别是最小支持度和数据集。

算法步骤：

1）生成所有单项（即项集长度为1）列表，得到满足最小支持度的1-项集<br>
2）将其进行组合成为2个元素的2-项集<br>
3）继续剔除不满足最小支持度的项集<br>
4）重复上述过程直到所有非频繁项集都被剔除。<br><br>
$Apriori$算法主要基于一种思想即如果某个项集是频繁项集，那么它的所有子集也是频繁的。即如果{0,1}是频繁的那么{0},{1}也一定是频繁的。<br>
反过来，也就是说一个项集如果是**非频繁项集**，那么它的所有超集也是非频繁的。

### 5.1.3 FP_Growth
虽然**Apriori**为挖掘频繁项集提供了很好的解决方案（通过筛选），但是**Apriori**通过不断构造筛选候选集挖掘出频繁项集，需要多次扫描原数据。因此当数据集比较庞大时，挖掘频繁项集的效率会十分低下。<br>
而**FP_Growth**（$Han$,$2000$）则只需要扫描原数据2遍，并将元数据压缩到一棵**FP-Tree**中，从而达到了压缩数据的目的。<br>
构造**FP-Tree**主要有2个步骤：从事务中构建**FP-Tree**和从**FP-Tree**中挖掘出规则。其具体步骤如下：
- 首先扫描数据集1次，生成1-频繁项集
- 将1-频繁项集降序排列好后放入$L$项频繁项集表中
- 再次扫描数据集，将每个事务相应项集的关联及频数计入**FP-Tree**中
- 确定好最小支持度计数，然后递归的挖掘频繁项集

接下来我将用例子来更好的说明这个算法，假设我们有如下数据集：

|$TID$|$Items$|
|----|----|
|$1$|$I_{1},I_{2},I_{5}$|
|$2$|$I_{2},I_{4}$|
|$3$|$I_{2},I_{3}$|
|$4$|$I_{1},I_{2},I_{4}$|
|$5$|$I_{1},I_{3}$|
|$6$|$I_{2},I_{3}$|
|$7$|$I_{1},I_{3}$|
|$8$|$I_{1},I_{2},I_{3},I_{5}$|
|$9$|$I_{1},I_{2},I_{3}$|

步骤一：我们需要计算每个项集的频数

|$I_{1}$|$I_{2}$|$I_{3}$|$I_{4}$|$I_{5}$|
|----|----|----|----|----|
|$6$|$7$|$6$|$2$|$2$|

步骤二：将1-频繁项集降序排列好后放入$L$项频繁项集表中


步骤三：再次扫描数据集，将每个事务相应项集的关联及频数计入**FP-Tree**中

步骤四：确定好最小支持度计数，然后递归的挖掘频繁项集<br>
此处拿$I_{3}$作为例子，设置最小支持度计数为1，得到前驱节点（也叫条件模式基）然后我们迭代的去找上面这种表中的节点，首先我们把$I_{2}$加入频繁项集队列，然后去找$I_{2}$的前驱节点发现不存在；于是我们接着把$I_{1}$加入频繁项集队列，然后去找$I_{1}$的前驱节点，所以此时频繁项集队列为

|||
|----|----|
|$I_1$|4|
|$I_2$|4|

然后我们继续递归，先把$I_{1},I_{2}$加入频繁项集队列（因为之前是以$I_{1}$作为基项的），然后发现$I_{2}$的前驱节点不存在所以对$I_{3}$的频繁项集挖掘到此结束<br>
我们就可以获得最后挖掘到的频繁项集

|||
|----|----|
|$I_1$|4|
|$I_2$|4|
|$I_{1},I_{2}$|2|

## 5.2 数据集
本次实验以经典的“啤酒-尿布”购物篮为例子，该数据集内容如下：<br>

|||
|----|----|
|$TID$|**item**|
|$1$|**bread,milk**|
|$2$|**bread,diaper,beer,egg**|
|$3$|**milk,diaper,beer,cola**|
|$4$|**bread,milk,diaper,beer**|
|$5$|**bread,milk,diaper,cola**|

# 6. 参考代码

### 步骤1 安装并引入必要的库

In [1]:
!pip install pandas
!pip install mlxtend

     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
      --------------------------------------- 0.0/1.4 MB 660.6 kB/s eta 0:00:03
     - -------------------------------------- 0.1/1.4 MB 409.6 kB/s eta 0:00:04
     --- ------------------------------------ 0.1/1.4 MB 726.2 kB/s eta 0:00:02
     ---- ----------------------------------- 0.1/1.4 MB 607.9 kB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.4 MB 737.3 kB/s eta 0:00:02
     ------ --------------------------------- 0.2/1.4 MB 801.7 kB/s eta 0:00:02
     ----------- ---------------------------- 0.4/1.4 MB 1.0 MB/s eta 0:00:01
     -------------- ------------------------- 0.5/1.4 MB 1.2 MB/s eta 0:00:01
     --------------- ------------------------ 0.5/1.4 MB 1.2 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.4 MB 1.4 MB/s eta 0:00:01
     ----------------------- ---------------- 0.8/1.4 MB 1.5 MB/s 

In [2]:
import pandas as pd
import mlxtend

### 步骤2 读取“啤酒与尿布”数据集

In [3]:
df = pd.read_csv('beer and diaper.csv')

In [4]:
df

,TID,item
0,1,"bread,milk"
1,2,"beer,diaper,bread,egg"
2,3,"beer,milk,cola,diaper"
3,4,"beer,milk,bread,diaper"
4,5,"milk,cola,bread,diaper"


### 步骤3 支持度计数

In [5]:
def is_subset(setA, setB):
    for i in setB:
        if i not in setA:
            return False
    return True

In [6]:
flag = df["item"].apply(is_subset, setB={"milk","diaper","beer"})
count = df[flag].shape[0]
print("Count is {}".format(count))

Count is 2


### 步骤4 支持度
求$"milk","diaper","beer"$的支持度

In [7]:
flag = df['item'].apply(is_subset, setB=({'milk', 'diaper', 'beer'}))
support = df[flag].shape[0]/df.shape[0]
print("support is {:.2f}".format(support))

support is 0.40


求$"milk","diaper"$的支持度

In [8]:
flag = df['item'].apply(is_subset, setB=({'milk', 'diaper'}))
support = df[flag].shape[0]/df.shape[0]
print("support is {:.2f}".format(support))

support is 0.60


### 步骤5 置信度
计算${milk,diaper}\rightarrow {beer}$的置信度

In [9]:
flag_ante = df["item"].apply(is_subset,setB=({'milk', 'diaper', 'beer'}))
count_ante = df[flag_ante].shape[0]
flag_conq = df["item"].apply(is_subset,setB=({'milk', 'diaper'}))
count_conq = df[flag_conq].shape[0]
conf = count_ante/count_conq
print("support is {:.2f}".format(conf))

support is 0.67


### 步骤6 Apriori算法

#### 6.1 创建数据集

将数据集转换成$one$-$hot$编码的形式

|$TID$|$bread$|$milk$|$egg$|$beer$|$diaper$|$cola$|
|----|----|----|----|----|----|----|
|$1$|$1$|$1$|$0$|$0$|$0$|$0$|
|$2$|$1$|$0$|$1$|$1$|$1$|$0$|
|$3$|$0$|$1$|$0$|$1$|$1$|$1$|
|$4$|$1$|$1$|$0$|$1$|$1$|$0$|
|$5$|$1$|$1$|$0$|$0$|$1$|$1$|

In [10]:
def count_item(itemset, item_name):
    items = itemset.split(',')
    count = 0
    for item in items:
        if item == item_name:
            count += 1
    return count

In [11]:
bread = df['item'].apply(count_item, item_name='bread')
milk = df['item'].apply(count_item, item_name='milk')
egg = df['item'].apply(count_item, item_name='egg')
beer = df['item'].apply(count_item, item_name='beer')
diaper = df['item'].apply(count_item, item_name='diaper')
cola = df['item'].apply(count_item, item_name='cola')
data = {'bread':bread,'milk':milk,'egg':egg,'beer':beer,'diaper':diaper,'cola':cola}
df_apriori = pd.DataFrame(data)
df_apriori

,bread,milk,egg,beer,diaper,cola
0,1,1,0,0,0,0
1,1,0,1,1,1,0
2,0,1,0,1,1,1
3,1,1,0,1,1,0
4,1,1,0,0,1,1


#### 6.2 使用Apriori算法

调用mlxtend包里的Apriori算法

**apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0, low_memory=False)**

从onehot形式的Dataframe计算频繁项集

**参数**

- df : pandas DataFrame

值可以是0/1或True/False

- min_support : float (default: 0.5)

最小支持度

- use_colnames : bool (default: False)

返回结果是否使用列名

- max_len : int (default: None)

允许的项集最大长度

- verbose : int (default: 0)

是否显示迭代轮数

- low_memory : bool (default: False)

是否使用节省内存模式

**返回值**

pandas.DataFrame

In [12]:
from mlxtend.frequent_patterns import apriori

apriori(df_apriori, min_support=0.5, use_colnames=True, verbose=1)

Processing 6 combinations | Sampling itemset size 3


C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.8,(bread)
1,0.8,(milk)
2,0.6,(beer)
3,0.8,(diaper)
4,0.6,"(milk, bread)"
5,0.6,"(diaper, bread)"
6,0.6,"(milk, diaper)"
7,0.6,"(beer, diaper)"


假设只对项集的长度大于2的感兴趣

使用pandas对apriori挖掘得到的频繁项集进行筛选

In [13]:
frequent_itemsets = apriori(df_apriori, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
0,0.8,(bread),1
1,0.8,(milk),1
2,0.6,(beer),1
3,0.8,(diaper),1
4,0.6,"(milk, bread)",2
5,0.6,"(diaper, bread)",2
6,0.6,"(milk, diaper)",2
7,0.6,"(beer, diaper)",2


In [14]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.5) ]

,support,itemsets,length
4,0.6,"(milk, bread)",2
5,0.6,"(diaper, bread)",2
6,0.6,"(milk, diaper)",2
7,0.6,"(beer, diaper)",2


### 步骤7 FP_Growth

下面来使用mlxtend库实现的FP_Growth算法

**fpgrowth(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0)**

从onehot形式的Dataframe计算频繁项集

参数

- df : pandas DataFrame

- min_support : float (default: 0.5)

- use_colnames : bool (default: False)

- max_len : int (default: None)

- verbose : int (default: 0)

是否显示详细过程

In [15]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df_apriori, min_support=0.5, use_colnames=True, verbose=1)

4 itemset(s) from tree conditioned on items ()
1 itemset(s) from tree conditioned on items (milk)
2 itemset(s) from tree conditioned on items (bread)
0 itemset(s) from tree conditioned on items (bread, milk)
0 itemset(s) from tree conditioned on items (bread, diaper)
0 itemset(s) from tree conditioned on items (diaper)
1 itemset(s) from tree conditioned on items (beer)


C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.8,(milk)
1,0.8,(bread)
2,0.8,(diaper)
3,0.6,(beer)
4,0.6,"(milk, diaper)"
5,0.6,"(milk, bread)"
6,0.6,"(diaper, bread)"
7,0.6,"(beer, diaper)"


### 步骤8 Apriori算法与FP_Growth算法的对比

使用timeit命令测试程序运行时间

`-n` 参数表示运行100次

`-r` 参数表示重复实验次数

In [16]:
%timeit -n 100 -r 10 apriori(df_apriori, min_support=0.6)

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

...

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

2.17 ms ± 83.3 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

In [17]:
%timeit -n 100 -r 10 apriori(df_apriori, min_support=0.6, low_memory=True)

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

...

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

2.29 ms ± 143 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

In [18]:
%timeit -n 100 -r 10 fpgrowth(df_apriori, min_support=0.6)

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: D

1.06 ms ± 41.6 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


可以看到FP-Growth算法要比Apriori算法运行更快

# 7. 习题
1. 在“啤酒与尿布”数据集中计算${bread}\rightarrow {milk}$的置信度
2. sklearn中哪个模块可以用来创建one-hot编码？（问答题）
3. movies.csv是近年来冯小刚导演的电影主演演员表，请用频繁模式挖掘的方法分析数据并回答，冯小刚导演喜欢合作的两位演员分别是谁？喜欢启用的演员组合是哪个？

In [19]:
flag_ante = df["item"].apply(is_subset,setB=({'bread', 'milk'}))
count_ante = df[flag_ante].shape[0]
flag_conq = df["item"].apply(is_subset,setB=({'bread'}))
count_conq = df[flag_conq].shape[0]
conf = count_ante/count_conq
print("support is {:.2f}".format(conf))

support is 0.75


# answer

可以使用`LabelBinarizer`和`MultiLabelBinarizer`两个类来实现，使用过程如下：

In [22]:
import numpy as np
# 先创建一个特征
nominal = np.array([["A"],
                   ["B"],
                   ["C"],
                   ["D"]])
# 导入LabelBinarizer
from sklearn.preprocessing import LabelBinarizer
one_hot = LabelBinarizer()  # 创建one-hot编码器
one_hot.fit_transform(nominal) # 对特征进行one-hot编码

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

也可以使用`OneHotEncoder`类来实现，使用过程如下：

In [25]:
from sklearn.preprocessing import  OneHotEncoder

enc = OneHotEncoder()
enc.fit([[0, 0, 3],
         [1, 1, 0],
         [0, 2, 1],
         [1, 0, 2]])

ans = enc.transform([[0, 1, 3]]).toarray()
ans


array([[1., 0., 0., 1., 0., 0., 0., 0., 1.]])

> **解释**
> 上述过程中成功将`[[0, 1, 3]]`转化为`[[1., 0., 0., 1., 0., 0., 0., 0., 1.]]`其中0转换为10，1转换为010，3转换为0001

In [32]:
dataf = pd.read_csv('movies.csv', encoding='gbk')
dataf

,芳华? (2017),天下无贼? (2004),唐山大地震? (2010),一九四二? (2012),甲方乙方? (1997),非诚勿扰? (2008),集结号? (2007),大腕? (2001),我不是潘金莲? (2016),非诚勿扰2? (2010),...,手机? (2003),不见不散? (1998),没完没了? (1999),一声叹息? (2000),一地鸡毛? (1995),手机2? (2020),永失我爱? (1994),月亮背面? (1997),跪族? (2007),情殇? (1995)
0,黄轩,刘德华,徐帆,张国立,葛优,葛优,张涵予,葛优,范冰冰,葛优,...,张国立,葛优,葛优,张国立,陈道明,葛优,徐帆,徐帆,范伟,赵宝刚
1,苗苗,刘若英,张静初,张默,刘蓓,舒淇,邓超,关之琳,郭涛,舒淇,...,葛优,徐帆,吴倩莲,刘蓓,徐帆,范冰冰,郭涛,冯远征,NaN,温海涛
2,钟楚曦,王宝强,李晨,徐帆,何冰,范伟,袁文康,英达,大鹏,孙红雷,...,范冰冰,NaN,傅彪,徐帆,修宗迪,张国立,剧雪,修宗迪,NaN,徐帆
3,杨采钰,李冰冰,陈道明,李雪健,英达,徐若瑄,汤嬿,保罗·马祖斯基,张嘉译,姚晨,...,徐帆,NaN,徐帆,傅彪,张瞳,徐帆,NaN,叶京,NaN,刘小宁
4,李晓峰,葛优,张子枫,陈道明,徐帆,方中信,廖凡,唐纳德·萨瑟兰,于和伟,安以轩,...,韩童生,NaN,刘蓓,李诚儒,周国治,范伟,NaN,梁丹妮,NaN,NaN
5,王天辰,张涵予,张家骏,艾德里安·布洛迪,杨立新,胡可,王宝强,李诚儒,张译,邵兵,...,黄素影,NaN,刘威,吴旭,徐秀林,NaN,NaN,马玉良,NaN,NaN
6,王可如,尤勇,王子文,蒂姆·罗宾斯,李琦,巩新亮,胡军,张涵予,李宗翰,廖凡,...,杨欣,NaN,韩童生,修宗迪,钟萍,NaN,NaN,秦焰,NaN,NaN
7,隋源,徐帆,陈瑾,冯远征,NaN,车晓,任泉,傅彪,赵立新,邬逸聪,...,张涵予,NaN,秦焰,高明,葛存壮,NaN,NaN,NaN,NaN,NaN


In [54]:
lst = []
for i in dataf:
    lst.append(list(dataf[i]))
enc.fit(lst)

dis = {}
for i in dataf:
#     print(dataf[i])
    for j in dataf[i]:
        if dis.get(j, None) == None:
            dis[j] = [list(dataf[k]).count(j) for k in dataf]
#     ans = enc.transform(tmp).toarray()
del dis[np.nan]
dap = pd.DataFrame(dis)


In [60]:
freq_it = apriori(dap, min_support=0.1, use_colnames=True, verbose = 1)
freq_it = freq_it.sort_values(by='support', ascending=False)
freq_it

Processing 30 combinations | Sampling itemset size 3


C:\Users\24636\.conda\envs\DataMining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.500000,( 葛优 )
2,0.500000,( 徐帆 )
4,0.181818,( 张国立 )
9,0.181818,"( 徐帆 , 葛优 )"
11,0.181818,"( 张国立 , 徐帆 )"
1,0.136364,( 张涵予 )
3,0.136364,( 陈道明 )
5,0.136364,( 刘蓓 )
6,0.136364,( 李诚儒 )
7,0.136364,( 范冰冰 )
